In [1]:
import torch
print(torch.__version__)

1.13.1+cpu


In [2]:
import torch
import torchvision.models as models
from torch.profiler import profile, ProfilerActivity

model = models.resnet18()
inputs = torch.randn(5, 3, 224, 224)

with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
   model(inputs)


In [3]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     aten::conv2d         0.07%     272.000us        78.32%     313.464ms      15.673ms            20  
                aten::convolution         0.20%     785.000us        78.26%     313.192ms      15.660ms            20  
               aten::_convolution         0.11%     446.000us        78.06%     312.407ms      15.620ms            20  
         aten::mkldnn_convolution        77.76%     311.219ms        77.95%     311.961ms      15.598ms            20  
                 aten::batch_norm         0.04%     174.000us         9.83%      39.354ms       1.968ms            20  
     aten::_batch_norm_impl_index       

In [4]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=30))


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls                                                                      Input Shapes  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                     aten::conv2d         0.01%      33.000us        17.87%      71.510ms      17.878ms             4                             [[5, 64, 56, 56], [64, 64, 3, 3], [], [], [], [], []]  
                aten::convolution         0.03%     131.000us        17.86%      71.477ms      17.869ms             4                     [[5, 64, 56, 56], [64, 64, 3, 3], [], [], [], [], [], 

In [5]:
prof.export_chrome_trace("trace.json")

In [7]:
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
   for i in range(10):
      model(inputs)
      prof.step()

In [8]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     aten::conv2d         0.05%       1.718ms        77.34%        2.779s      13.897ms           200  
                aten::convolution         0.18%       6.336ms        77.29%        2.778s      13.888ms           200  
               aten::_convolution         0.09%       3.228ms        77.11%        2.771s      13.857ms           200  
         aten::mkldnn_convolution        76.84%        2.762s        77.02%        2.768s      13.841ms           200  
                 aten::batch_norm         0.03%       1.209ms        11.16%     401.103ms       2.006ms           200  
     aten::_batch_norm_impl_index       

In [13]:
from torch.profiler import profile, tensorboard_trace_handler
with profile(activities=[ProfilerActivity.CPU], record_shapes=True, on_trace_ready=tensorboard_trace_handler("./log/resnet18")) as prof:
   for i in range(10):
      model(inputs)
      prof.step()

SyntaxError: cannot assign to operator (946937149.py, line 1)